In [4]:
!pip install torchtext==0.8.0
!pip freeze | grep torchtext

     |████████████████████████████████| 6.9MB 10.9MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
torchtext==0.8.0


In [5]:
!rm -rf *
!git config --global user.name "Akhilez"
!git config --global user.email "akhild18@yahoo.com"
!git clone https://github.com/Akhilez/ml_gallery.git
%cd ml_gallery/ml_py

Cloning into 'ml_gallery'...
remote: Enumerating objects: 787, done.
remote: Counting objects: 100% (787/787), done.
remote: Compressing objects: 100% (516/516), done.
remote: Total 3962 (delta 471), reused 553 (delta 256), pack-reused 3175
Receiving objects: 100% (3962/3962), 41.09 MiB | 40.81 MiB/s, done.
Resolving deltas: 100% (2428/2428), done.
/content/ml_gallery/ml_py


In [51]:
from google.colab import drive
drive.mount('/content/gdrive')
models_path = '/content/gdrive/MyDrive/Projects/ML/next_char'

In [6]:
import os
os.environ['SECRET_KEY'] = '1234'
from mlg.settings import BASE_DIR
os.environ['BASE'] = BASE_DIR
%mkdir -p ${BASE}/data/subtitles
%mkdir -p ${BASE}/models
!wget -O ${BASE}/data/subtitles/cleaned.txt https://storage.googleapis.com/akhilez/datasets/marvel_subtitles/cleaned.txt
!wget -O ${BASE}/data/subtitles/cleaned_test.txt https://storage.googleapis.com/akhilez/datasets/marvel_subtitles/cleaned_test.txt

--2020-12-06 05:01:40--  https://storage.googleapis.com/akhilez/datasets/marvel_subtitles/cleaned.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.80, 172.253.122.128, 172.217.7.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349823 (342K) [text/plain]
Saving to: ‘/content/ml_gallery/ml_py/data/subtitles/cleaned.txt’

/content/ml_gallery 100%[===================>] 341.62K  --.-KB/s    in 0.002s  

2020-12-06 05:01:40 (152 MB/s) - ‘/content/ml_gallery/ml_py/data/subtitles/cleaned.txt’ saved [349823/349823]

--2020-12-06 05:01:40--  https://storage.googleapis.com/akhilez/datasets/marvel_subtitles/cleaned_test.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.240, 172.217.15.80, 172.253.63.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.240|:443... connected.
HTTP request sent, awaiting response.

In [7]:
import torch
from torchtext.data import Field, TabularDataset, BucketIterator
from mlg.settings import BASE_DIR
from tqdm import tqdm
from torch import nn, optim
import torch.nn.functional as F
from datetime import datetime
from app.nlp.next_char.utils import save_vocab

In [62]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cleaned_data_path = f'{BASE_DIR}/data/subtitles/cleaned_test.txt'
data_path = f'{BASE_DIR}/data/subtitles'

batch_size = 64
seq_len = 64

pad_tkn = '~'
unk_tkn = '*'
eos_tkn = '\n'
init_tkn = '>'

cuda


In [63]:
TEXT = Field(sequential=True, tokenize=list, fix_length=seq_len, unk_token=unk_tkn, pad_first=False,
             pad_token=pad_tkn, eos_token=eos_tkn, init_token=init_tkn)

train_dataset, test_dataset = TabularDataset.splits(
    path=data_path,
    train='cleaned.txt', test='cleaned_test.txt',
    format='csv',
    skip_header=False,
    fields=[("text", TEXT)])

TEXT.build_vocab(train_dataset)
vocab_size = len(TEXT.vocab.itos)
torch.save(TEXT.vocab, f'{models_path}/vocab.pt')

train_iter, test_iter = BucketIterator.splits(
    (train_dataset, test_dataset),
    batch_sizes=(batch_size, batch_size),
    device=device,
    sort_key=lambda txt: len(txt.text),
    sort_within_batch=False,
    repeat=True
)


/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warn

In [69]:
class NextCharModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.embed = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=self.embed_size
        )

        self.rnn = nn.LSTM(
            input_size=self.embed_size,
            hidden_size=self.hidden_size
            #nonlinearity='relu'
        )

        self.y = nn.Linear(self.hidden_size, vocab_size)

    def forward(self, x):
        y = F.relu(self.embed(x))
        y, _ = self.rnn(y)
        return F.softmax(self.y(y), 2)


In [67]:
def load_model(latest=True, name=None):
    if latest:
        model_name = max(os.listdir(models_path))
        model = NextCharModel(vocab_size, 512, 512)
        model.load_state_dict(torch.load(f'{models_path}/{model_name}')).to(device)
        return model

In [81]:
model = NextCharModel(vocab_size, 512, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 5
model.train()
log_freq = 500
for epoch in range(epochs):
    i = 0
    losses = 0
    print(f'Epoch: {epoch}')
    for batch in train_iter:
        x_batch = batch.text
        y_batch = x_batch[1:]
        x_batch = x_batch[:-1]

        y_pred = model(x_batch)
        loss = loss_fn(y_pred.view((-1, vocab_size)), y_batch.flatten())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses += loss.item()

        i+=1
        if i % log_freq == 0:
            print(i / log_freq, losses / log_freq)
            losses = 0

        if i % (log_freq * 10) == 0:
            test_sentence = "Hey, wha"
            pred = predict(test_sentence)
            print(f'"{pred}"')
            model.train()

        if i % (log_freq * 50) == 0:
            print("Saving model")
            save_model(model, f'epoch{epoch}_batch{i}')

    save_model(model) 

Epoch: 0


/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


1.0 3.1701719732284546
2.0 3.1194721722602843
3.0 3.1162712798118593
4.0 3.115444986343384
5.0 3.1134845552444457
6.0 3.1117298069000245
7.0 3.10358247423172
8.0 3.099785491943359
9.0 3.0994732060432435
10.0 3.09595356464386
"Hey, what to to to to to to to to to to to to to t"
11.0 3.089969784259796
12.0 3.0891074357032777
13.0 3.0854917674064635
14.0 3.0835143847465516
15.0 3.0794866609573366
16.0 3.078847870826721
17.0 3.076128319263458
18.0 3.0746113185882566
19.0 3.0700501656532286
20.0 3.0656169443130494
"Hey, what i the the the the the the the the the th"
21.0 3.06476021528244
22.0 3.062280399799347
23.0 3.060857744693756
24.0 3.0605898542404173
25.0 3.053991011619568
26.0 3.046578258037567
27.0 3.044627574443817
28.0 3.042070695400238
29.0 3.0405831704139707
30.0 3.038519187927246
"Hey, what i stare the sare the sare the sare the s"
31.0 3.0373146562576294
32.0 3.0367769742012025
33.0 3.0354204034805297
34.0 3.035142090320587
35.0 3.0335770316123964
36.0 3.0335132336616515
37.0 

In [11]:

def predict(sentence):
    terminal_chars = [eos_tkn, '\n', pad_tkn]
    max_len = 50
    next_char = 0
    model.eval()
    with torch.no_grad():
        while next_char not in terminal_chars and len(sentence) < max_len:
            seq = torch.tensor([TEXT.vocab[s] or TEXT.vocab[unk_tkn] for s in list(sentence.lower())], device=device, dtype=torch.long).view((-1, 1))
            preds = model(seq)
            m = int(preds[-1][0].argmax())
            next_char = TEXT.vocab.itos[m]
            sentence = sentence + next_char
    return sentence


test_sentence = "Hey, what's u"

pred = predict(test_sentence)
print(f'"{pred}"')


"Hey, what's u~"


In [56]:
last_saved_model_path = ''
def save_model(model, message=None):
    last_saved_model_path = f'{models_path}/model_{int(datetime.now().timestamp())}{f"_{message}" if message else ""}.pt'
    torch.save(model.state_dict(), last_saved_model_path)
#save_model(model, message='test')

In [65]:
save_model(model, 'rnn')